In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

In [2]:
import cvxpy as cp
from measure_optimize import MeasurementOptimizer

(CVXPY) Aug 28 10:00:29 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')
(CVXPY) Aug 28 10:00:29 PM: Encountered unexpected exception importing solver GUROBI:
ImportError('libgurobi95.so: cannot open shared object file: No such file or directory')


## Input setting

### Read Q

In [3]:
jaco_info = pd.read_csv('./kinetics_fim/Q.csv')


# remove all initial conditions
Q_stack = []

for i in range(1,9):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(10,18):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
for i in range(19,27):
    
    Q_res = [jaco_info['A1'].iloc[i], jaco_info['A2'].iloc[i], jaco_info['E1'].iloc[i], jaco_info['E2'].iloc[i]]
    
    Q_stack.append(Q_res)
    
#print(Q_stack)   
print(np.shape(Q_stack))

(24, 4)


### Set Sigma

In [6]:
# variance 
variance = [1, # var_CA^2
            4, # var_CB^2
            8  # var_CC^2
           ]
covariance = [ 0.1, # cov_CA_CB | ti, 
               0.1, # cov_CA_CC | ti, 
               0.5 # cov_CB_CC | ti
]

#variance  = [1,1,1]
#covariance = [0,0,0]

# construct Sigma
sigma = np.zeros((24, 24))

for i in range(24):
    #for j in range(27):
        
        #if i==j:
    if i<8:
        sigma[i,i]=variance[0]
        sigma[i,i+8]=covariance[0]
        sigma[i,i+16]=covariance[1]
    elif 8<= i < 16:
        sigma[i,i]=variance[1]
        sigma[i,i-8]=covariance[0]
        sigma[i,i+8]=covariance[2]
    elif i>=16:
        sigma[i,i]=variance[2]
        sigma[i,i-16]=covariance[1]
        sigma[i,i-8]=covariance[2]
        
print(np.shape(sigma))

(24, 24)


### Other settings

In [7]:
number_measure = 3 
number_t = 8
cost = [10]*number_t + [6]*number_t*2 

## Check square problem solutions

In [8]:
#ya1 = [1, 1, 1, 1, 0, 0, 0, 0]
#yb1 = [1, 1, 1, 0.8895839685893835, 0, 1, 1, 1]
#yc1 = [1, 0.5873479124974504, 0, 0, 0, 0, 0.5230594984957986, 1]


#solution1 = ya1 + yb1 + yc1 
#solution1 = [0.9998548927512535, 0.6718489572976762, 0.6718489573007947, 0.9999473328860441, 0.6718489576894476, 0.6718489576645006, 0.6718489575876658, 0.6718489574880149, 0.43267878357849543, 0.9999125524850495, 0.9999125524850507, 0.5169740683761896, 0.9999125524850498, 0.9999125524850518, 0.9999125524850465, 0.9999125524850458, 0.9999666867834678, 0.9999125524850491, 0.9999125524850482, 0.9999013568405221, 0.9999125524850512, 0.9999125524850491, 0.9999125524850498, 0.9999125524850477]
solution1 = [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.8893433395380824, 0.0, 1.0, 1.0, 1.0, 1.0, 0.5875557934505837, 0.0, 0.0, 0.0, 0.0, 0.5230994268276815, 1.0]

In [9]:
calculator = MeasurementOptimizer(Q_stack, number_measure, number_t, cost, error_cov = sigma)

calculator.compute_FIM(solution1)

Q shape: 24 4
======FIM result======
FIM: [[  4.98298661   1.863857    -8.89915046  -7.76580571]
 [  1.863857     4.24420484  -2.72598598 -17.02395974]
 [ -8.89915046  -2.72598598  16.21349964  11.49775364]
 [ -7.76580571 -17.02395974  11.49775364  69.63693322]]
Determinant: 5.119105814581863 ; log_e(det): 1.6329797782096196 ; log_10(det): 0.7091941067360339
Trace: 95.07762430421698 ; log_e(trace): 4.554693655896879 ; log_10(trace): 1.9780783215157631
Min eig: 0.04722007437289387 ; log_e(min_eig): -3.052936172296492 ; log_10(min_eig): -1.3258733332312018
Cond: 1639.323491633256


array([[  4.98298661,   1.863857  ,  -8.89915046,  -7.76580571],
       [  1.863857  ,   4.24420484,  -2.72598598, -17.02395974],
       [ -8.89915046,  -2.72598598,  16.21349964,  11.49775364],
       [ -7.76580571, -17.02395974,  11.49775364,  69.63693322]])

## Optimize